In [18]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler

### Seed fixation

In [6]:
seed_value = 42

import os
os.environ['PYTHONHASHSEED'] = str(seed_value)

import random
random.seed(seed_value)

import numpy as np
np.random.seed(seed_value)

import tensorflow as tf
tf.random.set_seed(seed_value)

In [15]:
df = pd.read_csv("./data/valve1/0.csv", sep=";", parse_dates=True, index_col="datetime")
df = df.drop(["anomaly", "changepoint"], axis=1)
df.head()

,Accelerometer1RMS,Accelerometer2RMS,Current,Pressure,Temperature,Thermocouple,Voltage,Volume Flow RateRMS
datetime,,,,,,,,
2020-03-09 10:14:33,0.026588,0.040111,1.33020,0.054711,79.3366,26.0199,233.062,32.0
2020-03-09 10:14:34,0.026170,0.040453,1.35399,0.382638,79.5158,26.0258,236.040,32.0
2020-03-09 10:14:35,0.026199,0.039419,1.54006,0.710565,79.3756,26.0265,251.380,32.0
2020-03-09 10:14:36,0.026027,0.039641,1.33458,0.382638,79.6097,26.0393,234.392,32.0
2020-03-09 10:14:37,0.026290,0.040273,1.07851,-0.273216,79.6109,26.0420,225.342,32.0


### Make model

In [4]:
def create_dataset(df, win_size, gap_time, step_max):

    data = np.array(df, dtype=np.float64)
    sensor_n = data.shape[1]
    data_all = []
    scale_n = len(win_size)
    # min-max normalization
    data = MinMaxScaler().fit_transform(data).T

    for win in win_size:
        matrix_all = []
        print(f"generating signature with window {win} ...")
        for t in range(win_size[-1], len(df), gap_time):
            matrix_t = np.zeros((sensor_n, sensor_n))
            for i in range(sensor_n):
                for j in range(i, sensor_n):
                    matrix_t[i][j] = np.inner(data[i, t - win:t], data[j, t - win:t])/(win) # rescale by win
                    matrix_t[j][i] = matrix_t[i][j]
            matrix_all.append(matrix_t)
        data_all.append(matrix_all)

    data_all = np.transpose(data_all, (1,2,3,0))

    print (f"create dataset ...")

    dataset = data_all[:step_max]
    for i in range(step_max+1, len(data_all)+1):
        dataset = np.append(dataset, data_all[i-step_max:i], axis=0)

    return dataset.reshape([-1, step_max, sensor_n, sensor_n, scale_n])

In [30]:
win_sizes = [5, 10, 30]
gap_time = 1
h = 5

train_size = 400

dataset = create_dataset(df[:train_size], win_sizes, gap_time, h)

generating signature with window 5 ...
generating signature with window 10 ...
generating signature with window 30 ...
create dataset ...


In [22]:
import tensorflow as tf
from tensorflow import keras

In [49]:
from keras.models import Model
from keras.layers import Input, TimeDistributed, Conv2D, ConvLSTM2D, Conv2DTranspose, RepeatVector

In [56]:
class MSCRED:
    def __init__(self):
        pass
    
    def fit(self, train_dataset, batch_size=32, epochs=2):
        
        t, h, n, _, s = train_dataset.shape
        input_layer = Input(shape = (h, n, n, s))
        
        encoder_conv1 = TimeDistributed(Conv2D(filters = 32, kernel_size = 3, 
                               strides = 1, padding = "same",
                               activation="selu", kernel_initializer = "glorot_uniform"))(input_layer)
        
        encoder_conv2 = TimeDistributed(Conv2D(filters = 64, kernel_size = 3, 
                               strides = 2, padding = "same",
                               activation="selu", kernel_initializer = "glorot_uniform"))(encoder_conv1)
        
        encoder_conv3 = TimeDistributed(Conv2D(filters = 128, kernel_size = 2, 
                               strides = 2, padding = "same",
                               activation="selu", kernel_initializer = "glorot_uniform"))(encoder_conv2)
        
        encoder_conv4 = TimeDistributed(Conv2D(filters = 256, kernel_size = 2, 
                               strides = 2, padding = "same",
                               activation="selu", kernel_initializer = "glorot_uniform"))(encoder_conv3)
        
        print(encoder_conv4.shape)
        
        
        lstm_conv1 =  ConvLSTM2D(filters=32, kernel_size = 2, 
                                 padding = "same", return_sequences = False)(encoder_conv1)
        
        lstm_conv2 =  ConvLSTM2D(filters=64, kernel_size = 2, 
                                 padding = "same", return_sequences = False)(encoder_conv2)
        
        lstm_conv3 =  ConvLSTM2D(filters=128, kernel_size = 2, 
                                 padding = "same", return_sequences = False)(encoder_conv3)
        
        lstm_conv4 =  ConvLSTM2D(filters=256, kernel_size = 2, 
                                 padding = "same", return_sequences = False)(encoder_conv4)

        
        decoder_conv4 = Conv2DTranspose(filters=128, kernel_size = 2, strides = 2,
                                       kernel_initializer = "glorot_uniform", padding = "same",
                                       activation = "selu")(lstm_conv4)
        decoder4_out = tf.concat([decoder_conv4, lstm_conv3], axis = 3)
        
        decoder_conv3 = Conv2DTranspose(filters=64, kernel_size = 2, strides = 2,
                                       kernel_initializer = "glorot_uniform", padding = "same",
                                       activation = "selu")(decoder4_out)
        decoder3_out = tf.concat([decoder_conv3, lstm_conv2], axis = 3)
        
        decoder_conv2 = Conv2DTranspose(filters=32, kernel_size = 3, strides = 2,
                                       kernel_initializer = "glorot_uniform", padding = "same",
                                       activation = "selu")(decoder3_out)
        decoder2_out = tf.concat([decoder_conv2, lstm_conv1], axis = 3)
        
        decoder_conv1 = Conv2DTranspose(filters=s, kernel_size = 3, strides = 1,
                                       kernel_initializer = "glorot_uniform", padding = "same",
                                       activation = "selu")(decoder2_out)
        
        def loss_function(y_true, y_pred):
            return tf.reduce_mean(tf.square(y_true - y_pred))
        
        self.model = Model(input_layer, decoder_conv1)
        self.model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), 
                           loss=loss_function)
        
        self.model.fit(train_dataset, train_dataset, epochs=epochs, batch_size=batch_size, verbose=True)
        
        return self
    
    def predict(self, test_dataset):
        self.model.predict(test_dataset)

In [57]:
model = MSCRED().fit(dataset, epochs=10, batch_size=10)

(None, 5, 1, 1, 256)


ValueError: Input 0 of layer "repeat_vector_2" is incompatible with the layer: expected ndim=2, found ndim=4. Full shape received: (None, 8, 8, 32)

In [34]:
predictions = model.predict(create_dataset(df, win_sizes, gap_time, h))

generating signature with window 5 ...
generating signature with window 10 ...
generating signature with window 30 ...
create dataset ...
